In [ ]:
# Importamos librerias necesarias
import pandas as pd
import numpy as np
import seaborn as sns
import category_encoders as ce
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from IPython.display import clear_output
from sklearn.metrics import accuracy_score

In [ ]:
# Importamos el set de testeo provisto
inmuebles = pd.read_parquet("train.parquet")
testeo_henry = pd.read_parquet("test.parquet")

In [ ]:
# Revisamos las primeras lineas del dataset
inmuebles.head()


In [ ]:
# Revisamos existencia de valores nulos
inmuebles.isnull().sum()

In [ ]:
# Vamos a crear la nueva columna con las dos categorias de precios (1 = low, 0 = medium & high)
conditions = [
    (inmuebles['price'] >= 0) & (inmuebles['price'] <= 999),
    (inmuebles['price'] >= 1000)]
choices = ['1', '0']
inmuebles['category_price'] = np.select(conditions, choices).astype(int)

In [ ]:
inmuebles.info()

In [ ]:
# Verificamos valores de la nueva columna y existencia de nulos
inmuebles.category_price.unique()

In [ ]:
inmuebles.isnull().sum()

In [ ]:
# Graficamos la distribución de la nueva columna
sns.countplot(x='category_price', data=inmuebles)

Vemos que las clases se encuentran balanceadas

In [ ]:
# Verificamos existenvia de filas duplicadas
len(inmuebles)-len(inmuebles.drop_duplicates())

In [ ]:
inmuebles.shape

Explorando el dataset, encontramos Descripciones repetidas, generalmente con mismo precio de alquiler y mismo URL de la imagen de la propiedad, por lo que decidimos eliminar estos registros repetidos dejando solamente uno, utilizando para ello la columna de la URL de la imagen ya que es la que mayor confianza nos otorga en cuanto a que se trata de la misma propiedad en alquiler.

Consideramos que el dato de cuantas veces se publicó la misma propiedad puede ser un indicativo del valor de su alquiler. Por lo tanto, vamos a incorporar una columna con la cantidad de veces que se publicó el aviso.

In [ ]:
# Contamos la cantidad de valores repetidos en la columna "image_url"
len(inmuebles['image_url'])-len(inmuebles['image_url'].drop_duplicates())

Vemos que un número importante, practicamente la mitad del dataset contiene republicaciones de la misma propieadad en alquiler

In [ ]:
# Buscamos algun ejemplo para visualizar
df = inmuebles.loc[inmuebles['image_url'].duplicated(keep=False),'image_url']
df
df = df.groupby(df).apply(lambda x: tuple(x.index)).tolist()
df[24]

In [ ]:
inmuebles.loc[[66499,70514, 173295,215379,309342,331892],['price','image_url','description']]

Conciden los datos, con una pequeña variación en el costo del alquiler que puede deberse al paso del tiempo y la inflaciòn de Estados Unidos

In [ ]:
# Incorporamos una columna "publicaciones" que contiene la cantidad de veces que aparece el mismo anuncio en el dataset
inmuebles['publicaciones'] = inmuebles.groupby(['image_url'])['image_url'].transform('count')
testeo_henry['publicaciones'] = testeo_henry.groupby(['image_url'])['image_url'].transform('count')
inmuebles

In [ ]:
# Ahora si eliminamos los anuncios duplicados
inmuebles.drop_duplicates(subset='image_url', inplace=True)

In [ ]:
inmuebles.shape

Efectivamente, hemos reducido el dataset en un 51,4% del total de registros originales

In [ ]:
inmuebles

In [ ]:
# Volvemos a revisar la distribución del target
sns.countplot(x='category_price', data=inmuebles)

Vamos a analizar ahora cada columna en particular

In [ ]:
inmuebles.describe()

Notamos la existencia de outliers en varias columnas, por ejemplo, un valor máximo de 1.100 camas o 75 baños. Analizaremos a continuación cada una de las columnas

Analizemos "sqfeet" en busqueda de outliers

In [ ]:
inmuebles.sort_values("sqfeet",ascending=False)

Vemos que hay valores muy altos y valores en cero. De acuerdo a este artículo (https://www.ahs.com/home-matters/real-estate/the-2022-american-home-size-index/) el promedio de square feet de las viviendas en Estados Unidos se acerca de 2.000.
Por lo tanto, podemos considerar los valores superiores a 50.000 como errores (especialmente tratandose de alquileres).
Tambien consideraremos errores los valores de cero y muy bajos, ya que de acuerdo a este otro artículo (https://www.nyrentownsell.com/blog/square-footage-guide-to-living-real-life-home-example/) , 200 square feets es el mínimo para que la vivienda sea habitable
Calcularemos los outliers a continuacion utilizando el método del Rango Intercuartílico

In [ ]:
def outlier_treatment(datacolumn):
    """
    Definimos una función para detectar outliers
    """
    sorted(datacolumn)
    Q1,Q3 = np.percentile(datacolumn , [25,75])
    IQR = Q3 - Q1
    lower_range = Q1 - (1.5 * IQR)
    upper_range = Q3 + (1.5 * IQR)
    return lower_range,upper_range

In [ ]:
# Obtenemos los valores límites inferior y superior
lowerbound,upperbound = outlier_treatment(inmuebles.sqfeet)
print ("Lower:",lowerbound," ","Upper",upperbound)


In [ ]:
#Obtenemos algunos ejemplos de outliers
inmuebles[(inmuebles.sqfeet < lowerbound) | (inmuebles.sqfeet > upperbound)]

In [ ]:
# Graficamos
sns.boxplot(x="sqfeet", data=inmuebles)

In [ ]:
# Graficamos sin outliers
sns.boxplot(x = 'sqfeet', data = inmuebles, showfliers = False)

Como vemos en las gráficas, la cantidad de valores errones es importante. Por un lado, el valor mínimo de 75 squarefeet es muy bajo en relacion a los 200 squarefeet mencionados, con muchos casos dentro del primer cuartil con esos valores, y por otro lado, el valor mínimo de 1.875 es inferior al promedio del tamaño de viviendas en Estados Unidos.
Tomaremos entonces como valor mínimo válido 200 squarefeet y como valor máximo, 10.000 squarefeet, reemplazando los valores que exceden los límites, por dichos límites.

In [ ]:
inmuebles['sqfeet'].mask(inmuebles['sqfeet'] < 200, 200, inplace=True)
testeo_henry['sqfeet'].mask(testeo_henry['sqfeet'] < 200, 200, inplace=True)

In [ ]:
inmuebles['sqfeet'].mask(inmuebles['sqfeet'] > 10000, 10000, inplace=True)
testeo_henry['sqfeet'].mask(testeo_henry['sqfeet'] > 10000, 10000, inplace=True)

In [ ]:
# Volvemos a graficar
sns.boxplot(x="sqfeet", data=inmuebles)

Si bien a simple vista la gráfica indicaria la presencia abundante de outliers aún, consideramos que se relaciona satisfactoriamente a los datos recolectados. Una gran cantidad de propiedades en alquiler por debajo del tamaño promedio de las mismas en Estados Unidos, y una menor cantidad de propiedades de tamaños muy superiores a la media.

La columna "publicaciones" tiene valores (si bien reales), muy superiores a la media. Tomaremos como máximo 30 publicaciones. Ademas consideramos que el set de testeo, al contar con menos casos, no tendrá tantas repeticiones

In [ ]:
inmuebles['publicaciones'].mask(inmuebles['publicaciones'] > 30, 30, inplace=True)
testeo_henry['publicaciones'].mask(testeo_henry['publicaciones'] > 30, 30, inplace=True)

In [ ]:
# Graficamos
sns.boxplot( y=inmuebles["publicaciones"] )
plt.ylim(1, 30)
plt.show()

Veamos ahora las columnas de camas y baños, que parecen tener outliers

In [ ]:
# Graficamos camas
sns.boxplot(x="beds", data=inmuebles)

In [ ]:
# Ahora sin outliers
sns.boxplot(x = 'beds', data = inmuebles, showfliers = False)

Reemplazaremos los valores en 0 por 1 y los valores mayores a 8 por 8.

In [ ]:
inmuebles['beds'].mask(inmuebles['beds'] == 0, 1, inplace=True)
inmuebles['beds'].mask(inmuebles['beds'] > 8, 8, inplace=True)
testeo_henry['beds'].mask(testeo_henry['beds'] == 0, 1, inplace=True)
testeo_henry['beds'].mask(testeo_henry['beds'] > 8, 8, inplace=True)
sns.boxplot(x="beds", data=inmuebles)

In [ ]:
# Graficamos baños
sns.boxplot(x="baths", data=inmuebles)

Reemplazaremos los valores en 0 por 1 y los valores mayores a 8 por 8.

In [ ]:
inmuebles['baths'].mask(inmuebles['baths'] == 0, 1, inplace=True)
inmuebles['baths'].mask(inmuebles['baths'] > 8, 8, inplace=True)
testeo_henry['baths'].mask(testeo_henry['baths'] == 0, 1, inplace=True)
testeo_henry['baths'].mask(testeo_henry['baths'] > 8, 8, inplace=True)
sns.boxplot(x="baths", data=inmuebles)

Revisamos ahora la columna de Longitud. Vemos que hay valores en positivo que son erroneos, muchos casos son de Arkansas y vemos que si lo pasamos a negativo, coinciden con la ubicacion de ese Estado. Les cambiamos entonces el signo a los positivos y pasamos a -67.482 los valores mayores

In [ ]:
inmuebles["long"] = inmuebles["long"].abs()*(-1)
testeo_henry["long"] = testeo_henry["long"].abs()*(-1)

In [ ]:
inmuebles['long'].mask(inmuebles['long'] > (-67.482), (-67.482), inplace=True)
testeo_henry['long'].mask(testeo_henry['long'] > (-67.482), (-67.482), inplace=True)

In [ ]:
# Revisamos existencia de valores nulos
inmuebles["long"].isnull().sum()

In [ ]:
# Reemplazamos nulos por el promedio
inmuebles['long'].fillna(int(inmuebles['long'].mean()), inplace=True)
testeo_henry['long'].fillna(int(testeo_henry['long'].mean()), inplace=True)

Revisamos la columna de Servicio de Lavanderia. Vemos las opciones que contiene.

In [ ]:
inmuebles["laundry_options"].unique()

In [ ]:
inmuebles.laundry_options.isnull().sum()

Vemos que la cantidad de valores nulos es importante. Asumiremos que corresponden a que el inmueble no tiene el servicio

In [ ]:
inmuebles["laundry_options"].fillna("no laundry on site", inplace = True)
testeo_henry["laundry_options"].fillna("no laundry on site", inplace = True)
inmuebles.laundry_options.isnull().sum()

Convertimos la variable categorica en numérica. Lo hacemos de manera manual para ordenar los valores de acuerdo a la existencia o no del servicio

In [ ]:
numeric_var = {'laundry_options': {'no laundry on site':0, 'w/d hookups':1, 'laundry on site':2, 'laundry in bldg':3, "w/d in unit":4}}
inmuebles = inmuebles.replace(numeric_var)
testeo_henry = testeo_henry.replace(numeric_var)

Revisamos la columna de Estacionamiento. Vemos las opciones que contiene.

In [ ]:
inmuebles["parking_options"].unique()

In [ ]:
inmuebles.parking_options.isnull().sum()

Nuevamente, no podremos eliminar los nulos, y asumimos que corresponde a propiedad sin estacionamiento.
Reemplazamos los nulos y luego convertigos a valores numéricos, agrupando algunas categorias similares

In [ ]:
inmuebles["parking_options"].fillna("no parking", inplace = True)
testeo_henry["parking_options"].fillna("no parking", inplace = True)
inmuebles.parking_options.isnull().sum()

In [ ]:
numeric_var = {'parking_options': {'no parking':0, 'street parking':0, 'off-street parking':1, 'carport':2, "detached garage":2, "attached garage":3, "valet parking":4}}
inmuebles = inmuebles.replace(numeric_var)
testeo_henry = testeo_henry.replace(numeric_var)

Revisaremos ahora la correlación de longitud y latitud con el valor del alquiler para evaluar si eliminamos alguna de ellas o ambas

In [ ]:
correlaciones = inmuebles[["lat","long","category_price"]].copy()
correlaciones.corr()

Vemos que la longitud amerita participar del modelo debido a su correlación con el precio. Entendemos que obedece a que indica cercania con el mar.
Eliminaremos latitud, ademas de "price" que es un dato que no estará presente en el set de testeo de Henry, y las que creemos que no aportan valor o representan una gran dificultad para incorporar al modelo como "id", "url", "region", "region_url", "image_url" y "description"

In [ ]:
inmuebles.drop(["id","lat","url", "region", "region_url", "image_url","description","price"],axis = 1, inplace = True)
testeo_henry.drop(["id","lat","url", "region", "region_url", "image_url","description"],axis = 1, inplace = True)

Revisemos las correlaciones del estado actual del dataframe para eliminar algunas columnas adicionales que en principio no serían de utilidad

In [ ]:
# Graficamos)
plt.figure(figsize=(6, 10))
heatmap = sns.heatmap(inmuebles.corr(method='spearman')[['category_price']].sort_values(by='category_price', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Features Correlating with Category_Price', fontdict={'fontsize':18}, pad=16);


De acuerdo al gráfico, en un primer modelo usaremos beds, baths, parking options, laundry options, sqfeet, publicaciones, long y si se permite fumar

In [ ]:
# Vemos la distribución de la columna "type"
sns.countplot(y='type', data=inmuebles)

In [ ]:
# Veamos la relación entre el precio del alquiler y el tipo de vivienda
agr = inmuebles.groupby(['type', 'category_price']).size().reset_index().pivot(columns='category_price', index='type', values=0)
agr.plot(kind='bar', stacked=True)

Notamos que en las casas, condominios y townhouses la proporcion de alquileres mayores a $999 es mayor, por lo que en principio dejariamos esta columna. Transformamos a valores numéricos.

In [ ]:
numeric_var = {'type': {'land':0, 'in-law':0, 'assisted living':1, 'manufactured':1, "apartment":2, "flat":3, "townhouse":3, "loft":4 ,"duplex":4, "house":3, "cottage/cabin":4, "condo":5}}
inmuebles = inmuebles.replace(numeric_var)
testeo_henry = testeo_henry.replace(numeric_var)

In [ ]:
# Graficamos las nuevas categorias
sns.countplot(y='type', data=inmuebles)

In [ ]:
agr = inmuebles.groupby(['type', 'category_price']).size().reset_index().pivot(columns='category_price', index='type', values=0)
agr.plot(kind='bar', stacked=True)

Como vemos en este artículo online (https://www.fool.com/the-ascent/research/average-house-price-state/#:~:text=The%20median%20home%20price%20in,in%20the%20U.S.%20at%20%24354%2C649.), el Estado en el cual se ubican las propiedades tiene gran influencia sobre el precio promedio de las mismas.
Por lo tanto, vamos a utilizar la columna "state", pero necesitamos transformarla en valores numéricos.
Haremos esto mediante el método de Binary Encoding, ya que el One-Hot Encoding nos crearía demasiadas nuevas columnas

In [ ]:
inmuebles_ce = inmuebles.copy()
testeo_henry_ce = testeo_henry.copy()

encoder = ce.BinaryEncoder(cols=['state'])
inmuebles_binary = encoder.fit_transform(inmuebles_ce)
testeo_henry_binary = encoder.fit_transform(testeo_henry_ce)

inmuebles_binary.head()

In [ ]:
testeo_henry_binary.head()

In [ ]:
# Generamos un dataframe para hacer las primeras pruebas
inmuebles_v1 = inmuebles_binary
testeo_henry_v1 = testeo_henry_binary

In [ ]:
X = inmuebles_v1.drop(["cats_allowed","dogs_allowed","wheelchair_access", "electric_vehicle_charge", "comes_furnished", "category_price"],axis = 1)

In [ ]:
y = inmuebles_v1["category_price"]

Comenzaremos testeando un modelo de árbol de decisión

In [ ]:
tree_train_scores_mean = []
tree_train_scores_std = []
tree_test_scores_mean = []
tree_test_scores_std = []

profundidades = np.arange(1,50,1)
i = 0
i_max = len(profundidades)
for profundidad in profundidades:
    i = i + 1
    clf = DecisionTreeClassifier(max_depth=profundidad)
    tree_scores = cross_validate(clf, X, y, cv=5, return_train_score=True, n_jobs = -1)
    
    tree_train_scores_mean.append(tree_scores['train_score'].mean())
    tree_train_scores_std.append(tree_scores['train_score'].std())
    
    tree_test_scores_mean.append(tree_scores['test_score'].mean())
    tree_test_scores_std.append(tree_scores['test_score'].std())
    
    clear_output(wait=True)
    print('Completado: ' + str(round(i / i_max * 100, 2)) + '%')

tree_train_scores_mean = np.array(tree_train_scores_mean)
tree_train_scores_std = np.array(tree_train_scores_std)
tree_test_scores_mean = np.array(tree_test_scores_mean)
tree_test_scores_std = np.array(tree_test_scores_std)

In [ ]:
plt.fill_between(profundidades, tree_train_scores_mean - tree_train_scores_std,
                 tree_train_scores_mean + tree_train_scores_std, alpha=0.1,
                 color="r")
plt.fill_between(profundidades, tree_test_scores_mean - tree_test_scores_std,
                 tree_test_scores_mean + tree_test_scores_std, alpha=0.1, color="g")

plt.plot(profundidades, tree_train_scores_mean, 'o-', color="r",
         label="Training score")
plt.plot(profundidades, tree_test_scores_mean, 'o-', color="g",
         label="Test score")

plt.legend()
plt.ylabel('Accuracy')
plt.xlabel('Profundidad Arbol de Decision')
plt.show()

De acuerdo a la gráfica de las sucesivas pruebas, tomaremos como 27 la profundidad ideal del Árbol

In [ ]:
clf = DecisionTreeClassifier(max_depth = 27, random_state = 42) 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2,
    random_state=42, stratify=inmuebles_v1['category_price'])

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
print('Precisión en el set de entrenamiento: {0: .2f}'.format(clf.score(X_train, y_train)))
print('Precisión en el set de testeo: {0: .2f}'.format(clf.score(X_test, y_test)))

In [ ]:
X_test = testeo_henry_v1.drop(["cats_allowed","dogs_allowed","wheelchair_access", "electric_vehicle_charge", "comes_furnished"],axis = 1)
y_pred = clf.predict(X_test)

In [ ]:
y_pred

In [ ]:
pd.DataFrame(y_pred).to_csv('adelgerbo.csv', index=False, header = False)

# Entrega 1 --- .72 de accuracy y .68 de recall

Vamos a probar bajando a 12 la profundidad del Árbol

In [ ]:
clf = DecisionTreeClassifier(max_depth = 12, random_state = 42) 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2,
    random_state=42, stratify=inmuebles_v1['category_price'])

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
print('Precisión en el set de entrenamiento: {0: .2f}'.format(clf.score(X_train, y_train)))
print('Precisión en el set de testeo: {0: .2f}'.format(clf.score(X_test, y_test)))

In [ ]:
X_test = testeo_henry_v1.drop(["cats_allowed","dogs_allowed","wheelchair_access", "electric_vehicle_charge", "comes_furnished"],axis = 1)
y_pred = clf.predict(X_test)

In [ ]:
y_pred

In [ ]:
pd.DataFrame(y_pred).to_csv('adelgerbo.csv', index=False, header = False)

# Entrega 2 --- .74 de accuracy y .70 de recall

Vamos a probar eliminando las columnas del Estado

In [ ]:
X = X.drop(["state_0","state_1","state_2","state_3","state_4","state_5"], axis=1)

In [ ]:
tree_train_scores_mean = []
tree_train_scores_std = []
tree_test_scores_mean = []
tree_test_scores_std = []

profundidades = np.arange(1,30,1)
i = 0
i_max = len(profundidades)
for profundidad in profundidades:
    i = i + 1
    clf = DecisionTreeClassifier(max_depth=profundidad)
    tree_scores = cross_validate(clf, X, y, cv=5, return_train_score=True, n_jobs = -1)
    
    tree_train_scores_mean.append(tree_scores['train_score'].mean())
    tree_train_scores_std.append(tree_scores['train_score'].std())
    
    tree_test_scores_mean.append(tree_scores['test_score'].mean())
    tree_test_scores_std.append(tree_scores['test_score'].std())
    
    clear_output(wait=True)
    print('Completado: ' + str(round(i / i_max * 100, 2)) + '%')

tree_train_scores_mean = np.array(tree_train_scores_mean)
tree_train_scores_std = np.array(tree_train_scores_std)
tree_test_scores_mean = np.array(tree_test_scores_mean)
tree_test_scores_std = np.array(tree_test_scores_std)

In [ ]:
plt.fill_between(profundidades, tree_train_scores_mean - tree_train_scores_std,
                 tree_train_scores_mean + tree_train_scores_std, alpha=0.1,
                 color="r")
plt.fill_between(profundidades, tree_test_scores_mean - tree_test_scores_std,
                 tree_test_scores_mean + tree_test_scores_std, alpha=0.1, color="g")

plt.plot(profundidades, tree_train_scores_mean, 'o-', color="r",
         label="Training score")
plt.plot(profundidades, tree_test_scores_mean, 'o-', color="g",
         label="Test score")

plt.legend()
plt.ylabel('Accuracy')
plt.xlabel('Profundidad Arbol de Decision')
plt.show()

In [ ]:
clf = DecisionTreeClassifier(max_depth = 9, random_state = 42) 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2,
    random_state=42, stratify=inmuebles_v1['category_price'])

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
print('Precisión en el set de entrenamiento: {0: .2f}'.format(clf.score(X_train, y_train)))
print('Precisión en el set de testeo: {0: .2f}'.format(clf.score(X_test, y_test)))

In [ ]:
X_test = testeo_henry_v1.drop(["cats_allowed","dogs_allowed","wheelchair_access", "electric_vehicle_charge", "comes_furnished","state_0","state_1","state_2","state_3","state_4","state_5"],axis = 1)
y_pred = clf.predict(X_test)

In [ ]:
y_pred

In [ ]:
pd.DataFrame(y_pred).to_csv('adelgerbo.csv', index=False, header = False)

# Entrega 3 --- .79 de accuracy y .78 de recall

Vamos a revisar nuevamente las correlaciones para evaluar eliminar algunas columnas mas

In [ ]:
X

In [ ]:
inmuebles_v2 = inmuebles_v1.drop(["cats_allowed","dogs_allowed","wheelchair_access", "electric_vehicle_charge", "comes_furnished", "state_0","state_1","state_2","state_3","state_4","state_5"], axis=1)

In [ ]:
# Increase the size of the heatmap.
plt.figure(figsize=(16, 6))
# Store heatmap object in a variable to easily access it when you want to include more features (such as title).
# Set the range of values to be displayed on the colormap from -1 to 1, and set the annotation to True to display the correlation values on the heatmap.
heatmap = sns.heatmap(inmuebles_v2.corr(), vmin=-1, vmax=1, annot=True)
# Give a title to the heatmap. Pad defines the distance of the title from the top of the heatmap.
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

In [ ]:
plt.figure(figsize=(8, 12))
heatmap = sns.heatmap(inmuebles_v2.corr()[['category_price']].sort_values(by='category_price', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Features Correlating with Category_Price', fontdict={'fontsize':18}, pad=16);

Eliminaremos "Smoking Allowed", "Publicaciones", "Type y "Beds"

In [ ]:
X = X.drop(["smoking_allowed","publicaciones","type"], axis=1)

In [ ]:
tree_train_scores_mean = []
tree_train_scores_std = []
tree_test_scores_mean = []
tree_test_scores_std = []

profundidades = np.arange(1,30,1)
i = 0
i_max = len(profundidades)
for profundidad in profundidades:
    i = i + 1
    clf = DecisionTreeClassifier(max_depth=profundidad)
    tree_scores = cross_validate(clf, X, y, cv=5, return_train_score=True, n_jobs = -1)
    
    tree_train_scores_mean.append(tree_scores['train_score'].mean())
    tree_train_scores_std.append(tree_scores['train_score'].std())
    
    tree_test_scores_mean.append(tree_scores['test_score'].mean())
    tree_test_scores_std.append(tree_scores['test_score'].std())
    
    clear_output(wait=True)
    print('Completado: ' + str(round(i / i_max * 100, 2)) + '%')

tree_train_scores_mean = np.array(tree_train_scores_mean)
tree_train_scores_std = np.array(tree_train_scores_std)
tree_test_scores_mean = np.array(tree_test_scores_mean)
tree_test_scores_std = np.array(tree_test_scores_std)

In [ ]:
plt.fill_between(profundidades, tree_train_scores_mean - tree_train_scores_std,
                 tree_train_scores_mean + tree_train_scores_std, alpha=0.1,
                 color="r")
plt.fill_between(profundidades, tree_test_scores_mean - tree_test_scores_std,
                 tree_test_scores_mean + tree_test_scores_std, alpha=0.1, color="g")

plt.plot(profundidades, tree_train_scores_mean, 'o-', color="r",
         label="Training score")
plt.plot(profundidades, tree_test_scores_mean, 'o-', color="g",
         label="Test score")

plt.legend()
plt.ylabel('Accuracy')
plt.xlabel('Profundidad Arbol de Decision')
plt.show()

In [ ]:
clf = DecisionTreeClassifier(max_depth = 11, random_state = 42)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2,
    random_state=42) 
clf.fit(X_train, y_train)
print('Precisión en el set de entrenamiento: {0: .2f}'.format(clf.score(X_train, y_train)))
print('Precisión en el set de testeo: {0: .2f}'.format(clf.score(X_test, y_test)))

In [115]:
testeo_henry_v2 = testeo_henry_v1.drop(["cats_allowed","dogs_allowed","wheelchair_access", "electric_vehicle_charge", "comes_furnished", "state_0","state_1","state_2","state_3","state_4","state_5","smoking_allowed","publicaciones","type"], axis=1)

In [117]:
X_test = testeo_henry_v2
y_pred = clf.predict(X_test)

In [121]:
pd.DataFrame(y_pred).to_csv('adelgerbo.csv', index=False, header = False)

In [ ]:
# Exportamos para usar en el metodo no supervisado
X.to_csv("para_importar_no_supervisado_entrenamiento.csv", index=False)
X_test.to_csv("para_importar_no_supervisado_testeo_henry.csv", index=False)